# Đọc file final_report.csv

In [2]:
import pandas as pd

df = pd.read_csv("final_report.csv")

df["order_date"] = pd.to_datetime(df["order_date"])

df["month"] = df["order_date"].dt.to_period("M").astype(str)

# 1.Pivot doanh thu theo category × tháng

In [3]:
df["revenue"] = df["quantity"] * df["unit_price"]

pivot_revenue = pd.pivot_table(
    df,
    values="revenue",
    index="category",
    columns="month",
    aggfunc="sum",
    fill_value=0
)

pivot_revenue

month,2024-01,2024-02,2024-03,2024-04
category,,,,
gia dụng,5400000.0,10100000.0,21250000.0,2950000.0
điện tử,18950000.0,20100000.0,11950000.0,8450000.0


# 2.Pivot tỷ lệ trả hàng theo category × sản phẩm.

In [4]:
pivot_revenue = pd.pivot_table(
    df,
    values="revenue",       
    index="category",
    columns="month",
    aggfunc="sum",
    fill_value=0
)
pivot_return_rate = pd.pivot_table(
    df,
    values="is_returned",
    index="category",
    columns="product_name",
    aggfunc="mean"
)

KeyError: 'revenue'

# 3.Thực hiện stack/unstack bảng pivot.

In [ ]:
stacked = pivot_revenue.stack()
unstacked = stacked.unstack()


# 4.Nhận xét danh mục có hiệu suất thấp nhất.

In [ ]:
category_perf = pivot_revenue.sum(axis=1)

low_perf_category = category_perf.idxmin()

print("Danh mục có hiệu suất thấp nhất:", low_perf_category)

Danh mục có hiệu suất thấp nhất: điện tử
